# Setup

In [1]:
%cd ..

/home/pdona/VariousExercises/SentimentAnalysisAmazonMerchant


In [3]:
import pandas as pd
from nltk.corpus import stopwords
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
import re

### Clean the data

In [ ]:
# Define function to expand contractions
def expand_contractions(text):
    contractions_pattern = re.compile(
        f"({'|'.join(CONTRACTION_MAP.keys())})", flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = CONTRACTION_MAP.get(match)\
            if CONTRACTION_MAP.get(match)\
            else CONTRACTION_MAP.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


In [ ]:
def normalize_text(text: str) -> str:
    tmp_text = text.lower()
    tmp_text = expand_contractions(tmp_text)
    tmp_text = unicodedata.normalize('NFKD', tmp_text).encode(
        'ascii', 'ignore').decode('utf-8', 'ignore')
    tmp_text = re.sub(r'[^a-zA-z\s]', ' ', tmp_text)

    words = nltk.word_tokenize(tmp_text)

    stopword_list = stopwords.words('english')+['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
    stopword_list.remove('no')
    stopword_list.remove('not')
    words = [w for w in words if w not in stopword_list]

    # lemmatizer = WordNetLemmatizer()
    # lemmas = [lemmatizer.lemmatize(w, pos='v') for w in words]
    return ' '.join(words)


### Load data from database

In [3]:
df = pd.read_csv('SentimentAnalysis/data/AnkerAmazonReviews.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'merchant_token', 'merchant_name', 'product_asin',
       'product_name', 'product_average_review', 'review_user',
       'review_rating', 'review_title', 'review_date', 'review_verified',
       'review_content', 'review_helpful_vote', 'review_helpful',
       'review_length', 'review_polarity', 'review_subjectivity',
       'review_content_clean', 'review_title_clean',
       'review_content_clean_wotitle', 'review_title_clean_wotitle'],
      dtype='object')

In [12]:
df.dropna(inplace=True)

# Model definition - Bag of Words

TF-IDF aims to quantify the importance of a given word relative to other words in the document and in the corpus.

One tool we can use for doing this is called Bag of Words. BoW is a classical text representation technique

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
vectorizer = CountVectorizer()
BoW = vectorizer.fit_transform(list(df["review_content_clean"]))


In [34]:
vectorizer.get_feature_names_out()


array(['__', '___', '____', ..., 'zyxr', 'zz', 'zzzzzzzzzz'], dtype=object)

In [27]:
test = list(df["review_content_clean"].sample(5))

In [28]:
test

['got one wifes iphone xr really likes solid starts every time sets phone',
 'returned',
 'im liking job reading micro sd card like wanted im happy product',
 'charging well update time goes charging cables usually go months far holding well',
 'space saver cute design needed phone fit lamp diffuser big strip night stand ugly unnecessary perfect howver not charge phone via usb well super slow overcharge phone percent percent']

In [35]:
from textblob import classifiers

In [36]:
training = [
    ('Tom Holland is a terrible spiderman.', 'pos'),
    ('a terrible Javert (Russell Crowe) ruined Les Miserables for me...', 'pos'),
    ('The Dark Knight Rises is the greatest superhero movie ever!', 'neg'),
    ('Fantastic Four should have never been made.', 'pos'),
    ('Wes Anderson is my favorite director!', 'neg'),
    ('Captain America 2 is pretty awesome.', 'neg'),
    ('Let\s pretend "Batman and Robin" never happened..', 'pos'),
]
testing = [
    ('Superman was never an interesting character.', 'pos'),
    ('Fantastic Mr Fox is an awesome film!', 'neg'),
    ('Dragonball Evolution is simply terrible!!', 'pos')
]


In [37]:
classifier = classifiers.NaiveBayesClassifier(training)